In [ ]:
import torch
import torch.nn as nn ## importing neural network class from torch
import torch.optim as optim ## loads stochastic gradient descent and other functions
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Device Confirguration

In [ ]:
## check for CUDA availability -- cuda is a parallel processing software // use cuda architecture if the GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}") ## using Cuda -> output

Using cuda


#Data Collection & Processing


In [ ]:
# load the breast cancer dataset

data = load_breast_cancer()
X = data.data
y = data.target

#Split the Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
print(X.shape)
print(X_train.shape) ## approx 80% of data will be used for training the model
print(X_test.shape) ## approx 20% of the data will be used for r-valuation

(569, 30)
(455, 30)
(114, 30)


#Standardize Data

In [ ]:
## standardize the data // prevents data leakage -- StandardScalar (ensures the normal distribution of data)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
## check the type of the x_train and x_test data
type(X_train) # -> numpy array
type(X_test) # -> numpy array

numpy.ndarray

In [ ]:
# convert the data to PyTorch tensors and move it to the GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device) # -> moves the training data to the GPU after converting it to a PyTorch Tensor
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

<ipython-input-39-9bef028aae93>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.float32).to(device)


#Neural Network Architecture

In [ ]:
# inheriting the parameters from tne nn.Module to the custom class that is being defined

class neuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(neuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Fully connected layer
        self.relu = nn.ReLU()  # ReLU activation function
        self.fc2 = nn.Linear(hidden_size, output_size)  # Second fully connected layer
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for output

    def forward(self, x):
        out = self.fc1(x)  # Linear transformation (fully connected layer)
        out = self.relu(out)  # Apply ReLU activation
        out = self.fc2(out)  # Pass through second layer
        out = self.sigmoid(out)  # Apply Sigmoid to get final output
        return out


In [ ]:
# define the hyperparameters
input_size = X_train.shape[1] # -> represents the 30 features in the training data
hidden_size = 64
output_size = 1 # -> single neuron that gives the value between 0-1
learning_rate = 0.001
quant_epochs = 300


# Initialize the Neural Network

In [ ]:
# initialize the model
model = neuralNet(input_size, hidden_size, output_size).to(device) # moves the neural network to the device
# instance of the neural network is being defined

In [ ]:
# define the loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# training the model
for epoch in range(quant_epochs):
  model.train()
  optimizer.zero_grad() # -> each time a batch is passed, resets the gradients to 0 (prevents gradient accumulation)
  outputs = model(X_train)  # Get predictions for training data
  loss = criterion(outputs, y_train.view(-1,1)) # -> calculates the loss function with the predicted outputs
  loss.backward() # -> backward propogation
  optimizer.step() # -> updates the weights and bias based on the backwards proporgation

# calculate the model accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)

  if (epoch+1) % 10 == 0: ## for every 10th epoch, we are keeping track of the loss, accuracy, and the epoch
    print(f"Epoch [{epoch+1}/{quant_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")


Epoch [10/300], Loss: 0.5189, Accuracy: 89.23%
Epoch [20/300], Loss: 0.3960, Accuracy: 91.87%
Epoch [30/300], Loss: 0.3071, Accuracy: 92.97%
Epoch [40/300], Loss: 0.2453, Accuracy: 93.85%
Epoch [50/300], Loss: 0.2026, Accuracy: 94.51%
Epoch [60/300], Loss: 0.1723, Accuracy: 95.38%
Epoch [70/300], Loss: 0.1502, Accuracy: 95.82%
Epoch [80/300], Loss: 0.1335, Accuracy: 96.48%
Epoch [90/300], Loss: 0.1206, Accuracy: 97.14%
Epoch [100/300], Loss: 0.1103, Accuracy: 97.80%
Epoch [110/300], Loss: 0.1018, Accuracy: 97.80%
Epoch [120/300], Loss: 0.0948, Accuracy: 98.02%
Epoch [130/300], Loss: 0.0888, Accuracy: 98.24%
Epoch [140/300], Loss: 0.0836, Accuracy: 98.24%
Epoch [150/300], Loss: 0.0791, Accuracy: 98.24%
Epoch [160/300], Loss: 0.0751, Accuracy: 98.24%
Epoch [170/300], Loss: 0.0714, Accuracy: 98.24%
Epoch [180/300], Loss: 0.0682, Accuracy: 98.24%
Epoch [190/300], Loss: 0.0653, Accuracy: 98.24%
Epoch [200/300], Loss: 0.0627, Accuracy: 98.46%
Epoch [210/300], Loss: 0.0602, Accuracy: 98.46%
E

# Model Evaluation

In [ ]:
# evaluation on training set
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct/y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")


Accuracy on training data: 99.12%


In [ ]:
# evaluation on the test set
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct/y_test.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")


Accuracy on training data: 43.86%
